In [ ]:
import pypsa

import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import geopy.distance
import numpy as np

from collections import namedtuple

In [ ]:
%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('scripts/')

import simplify_network as simplify_network
import cluster_network as cluster_network

In [ ]:
eu_net = pypsa.Network("networks/elec.nc")

In [ ]:
n = pypsa.Network("../../singularity/carbon-flow/data/opf_ready_networks/Texas_breakthrough.nc")

In [ ]:
n.lines.v_nom.unique()

In [ ]:
n, map = simplify_network.simplify_network_to_380(n, newv=230)

In [ ]:
out_config = namedtuple('output', ["connection_costs"])("egcost.csv")


In [ ]:
n.generators["weight"] = np.nan

In [ ]:
# Remove generators with zero p_nom
for zerog in n.generators[n.generators.p_set==0].index:
    n.remove("Generator",zerog)

In [ ]:
def single_mode(s:pd.Series): 
    modes= s.mode()
    if len(modes)>0:
        return modes.loc[0]
    return None

# How should we aggregate each numeric column in generators? 
aggregation_strategies = {
    "generators":{
        "p_set":pd.Series.median,
        "q_set":pd.Series.median,
        "marginal_cost":pd.Series.median, 
        'q_max':pd.Series.median, 
        'q_min':pd.Series.median, 
        'v_set_pu':pd.Series.median, 
        'mva_base':pd.Series.median, 
       'p_min':pd.Series.max, 
       'Pc1':pd.Series.median, 
       'Pc2':pd.Series.median, 
       'Qc1min':pd.Series.median, 
       'Qc1max':pd.Series.median, 
       'Qc2min':pd.Series.median, 
       'Qc2max':pd.Series.median,
       'ramp_agc':pd.Series.median, 
       'ramp_10':pd.Series.median, 
       'ramp_30':pd.Series.median, 
       'ramp_q':pd.Series.median, 
       'apf':pd.Series.median, 
       'marginal_cost':pd.Series.median,
       'p_nom_min':pd.Series.sum, 
       'p_nom_max':pd.Series.sum, 
       'p_min_pu':pd.Series.median, 
       'p_max_pu':pd.Series.median, 
       'sign':pd.Series.mode, 
       'build_year':pd.Series.median,
       'lifetime':pd.Series.median, 
       'capital_cost':pd.Series.sum, 
       'efficiency':pd.Series.median, 
       'start_up_cost':pd.Series.sum,
       'shut_down_cost':pd.Series.sum, 
       'min_up_time':pd.Series.median, 
       'min_down_time':pd.Series.median, 
       'up_time_before':pd.Series.median,
       'down_time_before':pd.Series.median, 
       'ramp_limit_up':pd.Series.max, 
       'ramp_limit_down':pd.Series.max,
       'ramp_limit_start_up':pd.Series.median, 
       'ramp_limit_shut_down':pd.Series.median, 
       'p_nom_opt':pd.Series.median
    },
    "buses":{
        'Pd':pd.Series.median, 
        'Qd':pd.Series.median,
        'Gs':pd.Series.median,
        'Bs':pd.Series.median,
        'area':pd.Series.median,
        'v_mag_pu_set':pd.Series.median,
        'v_ang_set':pd.Series.median,
        'v_nom':pd.Series.median,
        'zone':single_mode, 
        'v_mag_pu_max':pd.Series.median,
        'v_mag_pu_min':pd.Series.median,
        'control':single_mode, 
        'x':pd.Series.median,
        'y':pd.Series.median,
        'type':single_mode,
        'carrier':single_mode,
        'unit':single_mode,
        'sub_network':single_mode,
        'country':single_mode,
        'generator':single_mode,
    }
}

In [ ]:
config = {"clustering":
        {"simplify_network":{
        "to_substations": False, # network is simplified to nodes with positive or negative power injection (i.e. substations or offwind connections)
        "algorithm": "kmeans", # choose from: [hac, kmeans]
        "exclude_carriers": []
    }}}

n, stub_map = simplify_network.remove_stubs(n, {}, config, out_config, aggregation_strategies)

In [ ]:
exclude_carriers = []
aggregate_carriers = set(n.generators.carrier) - set(exclude_carriers)

In [ ]:
n.buses["country"]="USA"

In [ ]:
# busmap for 
n.calculate_dependent_values()
len(n.buses.sub_network.unique())

In [ ]:
n.export_to_netcdf("networks/Texas_breakthrough_s.nc")

In [ ]:
N = int(len(n.buses)/5)
print(N)

In [ ]:
clustering = cluster_network.clustering_for_n_clusters(
    n, 
    128, # whole network subnet count
    #3169, # Eastern interconnect subnets
    False, # custom busmap?
    #n.buses.sub_network, # when clusters = n subnets  
    aggregate_carriers,
    1.25, # default from config.yaml
    aggregation_strategies,
    "ipopt",
    "kmeans",
    "", # shouldn't matter
    40.0, # HVAC overhead cost. not sure if correct
    None 
    )

In [ ]:
cluster_network.update_p_nom_max(clustering.network)

In [ ]:
clustering.network.export_to_netcdf("networks/Texas_breakthrough_s_128.nc")

In [ ]:
clustering.network.consistency_check()

In [ ]:
clustering.network.plot()

In [ ]:
n.plot()